In [1]:
%load_ext nb_mypy

Version 1.0.5


In [2]:
from loguru import logger
from s3path import S3Path
from datetime import datetime

from sm_pipelines_oo.steps.framework_processing_step import StepFactory
from sm_pipelines_oo.shared_config_schema import SharedConfig, Environment
from sm_pipelines_oo.aws_connector.concrete_connectors import create_aws_connector
from sm_pipelines_oo.config_loader.implementations.file_loaders import YamlConfigLoader

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/thomas-22/.config/sagemaker/config.yaml


In [3]:
ENV: Environment = 'local'
_config_loader = YamlConfigLoader(
    env=ENV,
    config_root_folder='../../config'  # relative to package root
)
_aws_connector = create_aws_connector(
    shared_config=SharedConfig(**_config_loader.shared_config_as_dict),
    environment=ENV,
)
factory = StepFactory(
    step_config_dict=_config_loader.step_configs_as_dicts[0],  # There is only 1 element
    role_arn=_aws_connector.role_arn,
    sm_session=_aws_connector.sm_session,
    pipeline_session=_aws_connector.pipeline_session,
)

In [4]:
from smp_oo_examples.worker_code.preprocess.preprocess import transform
from smp_oo_examples.worker_code.preprocess.validation import Validator

# Create input dataframe (with current date for validation) and save it to S3
_output_s3_dir = _config_loader.step_configs_as_dicts[0]['processor_run_config']['outputs']['output_1']
_output_filename = _config_loader.step_configs_as_dicts[0]['processor_init_config']['env']['OUTPUT_FILENAME']

validator = Validator(
    input_path_s3 = f"{_config_loader.step_configs_as_dicts[0]['processor_run_config']['inputs']['input_3']}/input.parquet",
    output_path_s3=f'{_output_s3_dir}/{_output_filename}',
    transform=transform
)

preprocess.py:  /home/thomas-22/repos/sagemaker-pipelines-abstraction/examples/src/smp_oo_examples


In [5]:
factory.run_processor()

INFO:sagemaker:Creating processing-job with name sklearn-2024-03-24-15-13-21-921
INFO:sagemaker.local.image:'Docker Compose' found using Docker CLI.
INFO:sagemaker.local.local_session:Starting processing job
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-gla77:
    container_name: 4msifma4ik-algo-1-gla77
    entrypoint:
    - /bin/bash
    - /opt/ml/processing/input/entrypoint/runproc.sh
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-gla77
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmp3g39f9l7/algo-1-gla77/config:/opt/ml/config
    - /tmp/tmp3g39f9l7/algo-1-gla77/output:/opt/ml/output
    - /tmp/tmppd3znz9i:/opt/ml/processing/input_3
    - /tmp/tmpax_b9z58:/opt/ml/processing/input/code/
    - /tmp

time="2024-03-24T11:13:26-04:00" level=warning msg="a network with name sagemaker-local exists but was not created for project \"tmp3g39f9l7\".\nSet `external: true` to use an existing network"
 Container 4msifma4ik-algo-1-gla77  Creating
 Container 4msifma4ik-algo-1-gla77  Created
Attaching to 4msifma4ik-algo-1-gla77
4msifma4ik-algo-1-gla77  | Found existing installation: typing 3.7.4.3
4msifma4ik-algo-1-gla77  | Uninstalling typing-3.7.4.3:
4msifma4ik-algo-1-gla77  |   Successfully uninstalled typing-3.7.4.3
4msifma4ik-algo-1-gla77  | Collecting typing_extensions (from -r requirements.txt (line 2))
4msifma4ik-algo-1-gla77  |   Downloading typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
4msifma4ik-algo-1-gla77  | Collecting awswrangler (from -r requirements.txt (line 3))
4msifma4ik-algo-1-gla77  |   Downloading awswrangler-3.7.1-py3-none-any.whl.metadata (23 kB)
4msifma4ik-algo-1-gla77  | Requirement already satisfied: boto3<2.0.0,>=1.20.32 in /miniconda3/lib/python3.8/sit

INFO:sagemaker.local.image:===== Job Complete =====


In [6]:
validator.validate_output()

Actual output:
   a  b                             date
0  2  4 2024-03-24 15:13:20.730195+00:00
1  2  4 2024-03-24 15:13:20.730195+00:00

Expected output:
   a  b                             date
0  2  4 2024-03-24 15:13:20.730195+00:00
1  2  4 2024-03-24 15:13:20.730195+00:00


In [7]:
print(datetime.now())

2024-03-24 11:13:39.086766
